In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
 
from datetime import datetime
def executed():
    #"%Y-%m-%d %I:%M:%S %p"
    print("excuted at", datetime.now().strftime("%Y-%m-%d %I:%M:%S")) 
executed()

excuted at 2024-01-29 05:10:11


BEGIN DATA PREP  
BEGIN USER-FACING PARAMETERIZATION STEP

In [3]:
csv = 'CVD.csv' #Pull data from csv for Anaconda development
categorical_features = ['x.Gender','Smoke']
numeric_features = ['x.Age']
binary_response = 'CVD' #Currently expects a 1/0-indicator for treatment/control, respectively
member_identifier = 'Unnamed: 0'  
exact_match_features = ['x.Gender','Smoke']

user_caliper = None #If left 'None', default caliper of 0.2*std.dev(logit) will be used.

Default caliper reference:  
Zhao QY, Luo JC, Su Y, Zhang YJ, Tu GW, Luo Z.  
Propensity score matching with R: conventional methods and new features.  
Ann Transl Med 2021;9(9):812.   
doi: 10.21037/atm-20-3998    

In [5]:
#Step 1: Backend set-up
covariates = categorical_features + numeric_features
df = pd.read_csv(csv)
X = df[covariates]
y = df[binary_response]
print("Step 1 excuted at", datetime.now().strftime("%Y-%m-%d %I:%M:%S"))                  

# Step 2: Encode Categorical Variables
# Include standardization in the pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])
print("Step 2 excuted at", datetime.now().strftime("%Y-%m-%d %I:%M:%S"))                  

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=21)
print("Step 3 excuted at", datetime.now().strftime("%Y-%m-%d %I:%M:%S"))                  

# Step 4: Build a logistic regression pipeline
logreg_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])
print("Step 4 excuted at", datetime.now().strftime("%Y-%m-%d %I:%M:%S"))                  

# Step 5: Fit the logistic regression model
logreg_model.fit(X_train, y_train)
print("Step 5 excuted at", datetime.now().strftime("%Y-%m-%d %I:%M:%S"))                  

# Step 6: Extract Propensity Scores for the entire dataset
propensity_scores = logreg_model.predict_proba(X)[:, 1]
print("Step 6 excuted at", datetime.now().strftime("%Y-%m-%d %I:%M:%S"))                  

#step 7: Create dataframe from logreg model and calculate logit 
result_df = pd.DataFrame({member_identifier: df[member_identifier], 'PropensityScore': propensity_scores})
result_df['logit'] = np.log(result_df['PropensityScore'] / (1-result_df['PropensityScore'])) 
print("Step 7 excuted at", datetime.now().strftime("%Y-%m-%d %I:%M:%S"))                  

#Step 8 Merge propensity scores and logit onto starting data set
df = pd.merge(df, result_df, on=member_identifier, how='left')
df['key']=1
print("Step 8 excuted at", datetime.now().strftime("%Y-%m-%d %I:%M:%S"))                  

Step 1 excuted at 2024-01-29 05:10:13
Step 2 excuted at 2024-01-29 05:10:13
Step 3 excuted at 2024-01-29 05:10:13
Step 4 excuted at 2024-01-29 05:10:13
Step 5 excuted at 2024-01-29 05:10:13
Step 6 excuted at 2024-01-29 05:10:13
Step 7 excuted at 2024-01-29 05:10:13
Step 8 excuted at 2024-01-29 05:10:13


END OF LOGISTIC REGRESSION AND SCORING  
BEGIN SUBSETTING (prior to KNN)

In [7]:
def generate_frequency_table(df, feature_combination):
    return df.groupby(feature_combination).size().reset_index(name='Frequency')

def subset_df_by_frequency_table(df, frequency_table):
    subsets = []
    for _, row in frequency_table.iterrows():
        subset = df.copy()
        for feature, value in zip(frequency_table.columns[:-1], row[:-1]):
            subset = subset[subset[feature] == value]
        subsets.append(subset)
    return subsets
executed()

excuted at 2024-01-29 05:10:13


In [8]:
frequency_table = generate_frequency_table(df=df, feature_combination=exact_match_features)
subsets = subset_df_by_frequency_table(df=df, frequency_table=frequency_table)

#del frequency_table
executed()

excuted at 2024-01-29 05:10:13


In [9]:
for x in subsets:
    print(x.groupby([binary_response]).size().reset_index(name='Frequency'))

   CVD  Frequency
0    0        255
1    1         32
   CVD  Frequency
0    0         67
1    1         46
   CVD  Frequency
0    0        188
1    1         67
   CVD  Frequency
0    0        187
1    1        158


In [10]:
def unique_knn(df, member_identifier):
    # Create an empty set to store unique 'mb_minority' and 'mb_majority' values
    unique_mb_minority = set()
    unique_mb_majority = set()

    # Create a list to store the indices to be dropped
    indices_to_drop = []

    # Iterate through the rows of the DataFrame
    for index, row in df.iterrows():
        mb_minority = row[member_identifier+'_min']
        mb_majority = row[member_identifier+'_maj']

        # Check if 'mb_minority' or 'mb_majority' is not in the respective set
        if mb_minority not in unique_mb_minority and mb_majority not in unique_mb_majority:
            # If neither 'mb_minority' nor 'mb_majority' have occurred before, add them to the sets
            unique_mb_minority.add(mb_minority)
            unique_mb_majority.add(mb_majority)
        else:
            # If either 'mb_minority' or 'mb_majority' has occurred before, mark the index to be dropped
            indices_to_drop.append(index)

    # Drop the marked indices from the DataFrame
    df.drop(indices_to_drop, inplace=True)

    # Reset the index of the resulting DataFrame
    df.reset_index(drop=True, inplace=True)

    return df

In [11]:
unique_knn_list=[]

for exact_subset_df in subsets:
    std_dev_score = exact_subset_df['logit'].std()
    caliper = user_caliper if user_caliper is not None else 0.2 * std_dev_score
    del std_dev_score
    df_status_0 = exact_subset_df[exact_subset_df[binary_response] == 0][[member_identifier,'PropensityScore','logit','key']]
    df_status_1 = exact_subset_df[exact_subset_df[binary_response] == 1][[member_identifier,'PropensityScore','logit','key']]

    if len(df_status_1) <= len(df_status_0):
        minority=df_status_1 
        majority=df_status_0
    else:
        minority=df_status_0 
        majority=df_status_1 
    subset_result=majority.merge(minority,on='key',suffixes=('_maj','_min')).drop('key',axis=1)
    del majority
    del minority
    subset_result['euclidean'] = ((subset_result['PropensityScore_min'] - subset_result['PropensityScore_maj'])**2)**0.5
    subset_result['caliper']=caliper
    subset_result = subset_result[[member_identifier+'_min', member_identifier+'_maj', 'PropensityScore_min', 'PropensityScore_maj', 'logit_min','logit_maj','euclidean','caliper']].query('euclidean < caliper').sort_values(by=[member_identifier+'_min', 'euclidean'], ascending=[True, True])
    unique_knn_list.append(unique_knn(df=subset_result, member_identifier=member_identifier))
    del subset_result
executed()

excuted at 2024-01-29 05:10:15


In [12]:
unique_knn = pd.concat(unique_knn_list, ignore_index=True)
#del unique_knn_list
executed()

excuted at 2024-01-29 05:10:15


In [13]:
unique_knn['match_id']= range(1, len(unique_knn) + 1) 
lkup_min = unique_knn[[member_identifier+'_min', 'match_id']].rename(columns={member_identifier+'_min': member_identifier})
lkup_maj = unique_knn[[member_identifier+'_maj', 'match_id']].rename(columns={member_identifier+'_maj': member_identifier})
executed()

excuted at 2024-01-29 05:10:15


In [14]:
lkup = pd.concat([lkup_min, lkup_maj]).sort_values(by=['match_id', member_identifier], ascending=[True, True]).reset_index(drop=True)
lkup['match_ind']=1
#del lkup_min
#del lkup_maj

In [15]:
# Merge 'df' with 'lkup' using a left join to retain all rows in 'df'
df = pd.merge(df, lkup, on= member_identifier, how='left')

In [16]:
print(df.head())

   Unnamed: 0  x.Age  x.Gender  Smoke  CVD  PropensityScore     logit  key  \
0           0   52.0         0      1    1         0.569969  0.281723    1   
1           1   43.0         0      1    1         0.129403 -1.906250    1   
2           2   55.0         0      0    0         0.076879 -2.485521    1   
3           3   68.0         0      0    1         0.662596  0.674884    1   
4           4   41.0         0      1    0         0.083749 -2.392466    1   

   match_id  match_ind  
0      33.0        1.0  
1      34.0        1.0  
2       NaN        NaN  
3       1.0        1.0  
4      61.0        1.0  


In [34]:
df.to_csv('p010_final.csv', index=False)